In [1]:
# Evaluate model predictions on "val" set

#from ultralytics import YOLO
import os
import pandas as pd
import yaml
import matplotlib.pyplot as plt

# Specify the directory containing your text files
text_files_dir = "/home/cara/oregon_critters/runs/detect/0_mc/val/labels"

# Initialize an empty list to store the data
data = []

# Iterate through each text file in the directory
for file_name in os.listdir(text_files_dir):
    file_path = os.path.join(text_files_dir, file_name)
    with open(file_path, "r") as file:
        lines = file.readlines()
        
        # Process each line (row) in the file
        for line in lines:
            values = line.strip().split()
            data.append([file_name] + values)

# Create dataframe 
columns = ["filename"] + [f"value_{i+1}" for i in range(len(data[0])-1)]
preds = pd.DataFrame(data, columns=columns)

# Create column names
column_names = ["image_name"] + ["class_ID"] + ["x"] + ["y"]+ ["w"]+ ["h"]+ ["confidence"]
preds.columns = column_names

# Remove .txt from image name
preds['image_name'] = preds.image_name.map(lambda x: x.strip('.txt'))
preds.head()



,image_name,class_ID,x,y,w,h,confidence
0,7762-4__2019-08-13__09-17-47(1),39,0.485311,0.377945,0.192238,0.104651,0.73672
1,7762-4__2019-08-13__09-17-47(1),26,0.760668,0.566443,0.0751886,0.0608607,0.209668
2,7762-4__2019-08-13__09-17-47(1),36,0.482066,0.377622,0.181581,0.105902,0.18962
3,7762-4__2019-08-13__09-17-48(2),17,0.62162,0.442645,0.0652517,0.12387,0.484186
4,7762-4__2019-08-13__09-17-48(2),26,0.76422,0.566579,0.0739105,0.0559245,0.186659


In [2]:
# Match class IDs with actual class names

# Read in yaml file
yaml_file_path = "/home/cara/oregon_critters/dataset.yaml"
with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Extract just the "names" data from the yaml and create list of dictionaries
names_data = yaml_data.get('names', {})
data = [{'class_ID': key, 'class_pred': value} for key, value in names_data.items()]

# Create dataframe
class_ids = pd.DataFrame(data)

# Convert column 'class_ID' to string
class_ids['class_ID'] = class_ids['class_ID'].astype(str)

# Merge with "preds" dataframe
preds=preds.merge(class_ids, how='left', on='class_ID')
preds.head()

# Save
preds.to_csv('/home/cara/oregon_critters/runs/detect/0_mc/pred_boxes_0mc.csv')


In [ ]:
## Skip this next part...

# Now read in metadata
meta_csv = pd.read_csv('/home/cara/oregon_critters/sampled_ds_300_split.csv')

meta_df = meta_csv[['IName', 'CName', 'cam_type', 'day_night', 'project', 'group']]
meta_df['IName'] = meta_df.IName.map(lambda x: x.strip('.JPG'))

#rename column
meta_df.rename(columns={"CName": "class_true"}, inplace=True)
meta_df.head()

# And combine dataframes
preds=preds.merge(meta_df, how='left', left_on='image_name', right_on = 'IName')
preds.head()

preds.to_csv('/home/cara/oregon_critters/runs/detect/0_mc/predictions_0mc.csv')


In [ ]:
# Match up true/predicted species classes
preds.class_match
preds.loc[preds['class_pred'] == preds['class_true'], 'class_match'] = 'True'
preds.head()

In [ ]:
# Now make histograms of scores

#all scores
plt.hist(preds['confidence'])
plt.xlabel('Species class')
plt.ylabel('Number of boxes')
plt.title('Confidence scores on validation set')

#color by true/false pos

#by species
